In [2]:
#!pip install matplotlib
#!pip install numpy
#!pip install pandas
#!pip install seaborn
#!pip install sklearn
#!pip install pandasql
#!pip install psycopg2
#!pip install pprint
#!pip install mlflow
#!pip install catboost
#!pip install boto3

In [2]:
import os
import warnings
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from pandasql import sqldf
from azure.storage.blob import BlockBlobService
import datetime as dt
from datetime import date
import re
from io import StringIO
import psycopg2
import pandas as pd
from sqlalchemy import create_engine,text
from typing import Optional
import os
import mlflow
from sklearn import metrics
from pprint import pprint
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,BayesianRidge
import math
import boto3
from datetime import datetime

#import ElasticNet

# %matplotlib inline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging
from sklearn import neighbors
from sklearn import svm
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import catboost
from catboost import Pool
from catboost import CatBoostRegressor




#logging.basicConfig(level=logging.WARN)
#logger = logging.getLogger(__name__)
#os.system("pip install boto3 mlflow")

In [3]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['MLFLOW_BASE_URL'] = ''

# connect to remote server
mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "cust-lifetime"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [4]:
pd.set_option('display.max_columns', 100)

In [5]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD


def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()

def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()

In [6]:
MODEL_NAME = "customer-lifetime-value"
POSTGRES_ADDRESS = ' ' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = ' '
POSTGRES_USERNAME = ' ' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = ' ' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = ' '
PROJECT_NAME = 'katonic'
PROJECT_SCHEMA = 'usecase'
TABLE_DATA = 'customer_lifetime_value_pre'
TABLE_MODEL = 'customer_lifetime_value_model'

In [7]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
X_df = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_DATA)
X_df

,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,location code,marital status,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size
0,600,4,2763.519279,0,0,0,47,1,0,56274,1,1,69,32,5,0,1,0,2,0,0,384.811147,5,1
1,5946,0,6979.535903,0,1,0,24,4,0,0,1,2,94,13,42,0,8,1,5,2,0,1131.464935,0,1
2,96,2,12887.431650,0,2,0,41,1,0,48767,1,1,108,18,38,0,2,1,5,0,0,566.472247,5,1
3,8016,1,7645.861827,0,0,0,12,4,1,0,1,1,106,18,65,0,7,0,1,0,2,529.881344,3,1
4,2488,4,2813.692575,0,0,0,52,1,1,43836,0,2,73,12,44,0,1,1,3,0,0,138.130879,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,3857,1,23405.987980,0,0,0,32,1,1,71941,2,1,73,18,89,0,2,1,3,1,3,198.234764,0,1
9130,5390,1,3096.511217,1,1,1,34,1,0,21604,1,0,79,14,28,0,1,0,2,0,1,379.200000,0,1
9131,6688,1,8163.890428,0,1,0,55,4,1,0,1,2,85,9,37,3,2,0,1,0,1,790.784983,0,1
9132,7214,1,7524.442436,0,1,1,52,1,1,21941,1,1,96,34,3,0,3,1,4,2,1,691.200000,0,0


In [8]:
df_1 = X_df.iloc[:1000,:]
df_2 = X_df.iloc[1001:,:]

In [9]:
df_1

,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,location code,marital status,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size
0,600,4,2763.519279,0,0,0,47,1,0,56274,1,1,69,32,5,0,1,0,2,0,0,384.811147,5,1
1,5946,0,6979.535903,0,1,0,24,4,0,0,1,2,94,13,42,0,8,1,5,2,0,1131.464935,0,1
2,96,2,12887.431650,0,2,0,41,1,0,48767,1,1,108,18,38,0,2,1,5,0,0,566.472247,5,1
3,8016,1,7645.861827,0,0,0,12,4,1,0,1,1,106,18,65,0,7,0,1,0,2,529.881344,3,1
4,2488,4,2813.692575,0,0,0,52,1,1,43836,0,2,73,12,44,0,1,1,3,0,0,138.130879,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7359,3,5595.389905,1,0,3,54,1,0,74454,1,2,71,25,29,0,3,1,4,1,2,340.800000,0,1
996,5529,3,4170.687348,1,0,4,8,1,0,29462,1,2,107,28,23,1,1,1,4,1,1,513.600000,3,0
997,7032,0,2665.447079,0,0,1,46,1,0,52266,0,2,68,14,98,0,1,0,0,0,2,141.922839,5,1
998,6904,3,7098.914140,0,0,0,56,4,1,0,1,2,70,18,99,0,2,1,3,0,0,349.783046,5,1


In [10]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,df_1,"table_val")

In [11]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
VAL = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,"table_val").iloc[::-1].reset_index(drop=True)
VAL.head()

,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,income,location code,marital status,monthly premium auto,months since last claim,months since policy inception,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size
0,9065,1,3971.345097,0,0,2,49,1,1,23599,1,2,103,20,46,0,1,0,2,1,0,494.400000,3,1
1,6904,3,7098.914140,0,0,0,56,4,1,0,1,2,70,18,99,0,2,1,3,0,0,349.783046,5,1
2,7032,0,2665.447079,0,0,1,46,1,0,52266,0,2,68,14,98,0,1,0,0,0,2,141.922839,5,1
3,5529,3,4170.687348,1,0,4,8,1,0,29462,1,2,107,28,23,1,1,1,4,1,1,513.600000,3,0
4,7359,3,5595.389905,1,0,3,54,1,0,74454,1,2,71,25,29,0,3,1,4,1,2,340.800000,0,1


In [12]:
Y_df = df_2['customer lifetime value']
df_2.pop('customer lifetime value')
Y_df.head()

1001     8338.995787
1002    38445.855870
1003     5448.555219
1004    10808.065970
1005     6185.096530
Name: customer lifetime value, dtype: float64

In [13]:
X = df_2

y = Y_df

In [14]:
X.columns

Index(['customer', 'state', 'response', 'coverage', 'education',
       'effective to date', 'employmentstatus', 'gender', 'income',
       'location code', 'marital status', 'monthly premium auto',
       'months since last claim', 'months since policy inception',
       'number of open complaints', 'number of policies', 'policy type',
       'policy', 'renew offer type', 'sales channel', 'total claim amount',
       'vehicle class', 'vehicle size'],
      dtype='object')

In [15]:
X_train = X.drop(['customer'], axis=1)
y_train = y

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.20,random_state=42)

In [16]:
X_test.columns

Index(['state', 'response', 'coverage', 'education', 'effective to date',
       'employmentstatus', 'gender', 'income', 'location code',
       'marital status', 'monthly premium auto', 'months since last claim',
       'months since policy inception', 'number of open complaints',
       'number of policies', 'policy type', 'policy', 'renew offer type',
       'sales channel', 'total claim amount', 'vehicle class', 'vehicle size'],
      dtype='object')

In [17]:
y_train

7366    17776.822670
8667     3531.348165
7568     2563.582441
2150    15886.031360
2872     6751.437153
            ...     
6227     9165.721087
6391     6135.083838
1861    37111.659520
8604     5300.686394
8271    16979.797570
Name: customer lifetime value, Length: 6506, dtype: float64

In [18]:
y_test

2303    33873.181230
2559     2842.025913
6975     9384.654502
6295     3600.934326
4843    16979.797570
            ...     
1294     5917.466603
6947    13448.809730
8589    10408.968650
5221     8923.154803
7688     6044.020038
Name: customer lifetime value, Length: 1627, dtype: float64

In [19]:
X_train.columns

Index(['state', 'response', 'coverage', 'education', 'effective to date',
       'employmentstatus', 'gender', 'income', 'location code',
       'marital status', 'monthly premium auto', 'months since last claim',
       'months since policy inception', 'number of open complaints',
       'number of policies', 'policy type', 'policy', 'renew offer type',
       'sales channel', 'total claim amount', 'vehicle class', 'vehicle size'],
      dtype='object')

In [20]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression
#Select top 2 features based on mutual info regression
selector = SelectKBest(mutual_info_regression, k =3)
selector.fit(X, y)
X.columns[selector.get_support()]



Index(['income', 'monthly premium auto', 'total claim amount'], dtype='object')

In [21]:
k_best_features = list(X.columns[selector.get_support(indices = True)])
k_best_features


['income', 'monthly premium auto', 'total claim amount']

In [22]:
X_train[k_best_features]

,income,monthly premium auto,total claim amount
7366,86205,223,1070.400000
8667,0,98,958.502743
7568,0,69,331.200000
2150,56362,197,945.600000
2872,0,95,456.000000
...,...,...,...
6227,53792,115,552.000000
6391,43579,77,452.095421
1861,61552,115,12.252847
8604,26585,69,308.651195


In [23]:
X_train, X_test = X_train[k_best_features], X_test[k_best_features]


In [24]:
X_train

,income,monthly premium auto,total claim amount
7366,86205,223,1070.400000
8667,0,98,958.502743
7568,0,69,331.200000
2150,56362,197,945.600000
2872,0,95,456.000000
...,...,...,...
6227,53792,115,552.000000
6391,43579,77,452.095421
1861,61552,115,12.252847
8604,26585,69,308.651195


In [25]:
#X = X_df['monthly premium auto'].values.reshape(-1,1)
#y = X_df['customer lifetime value'].values.reshape(-1,1)

In [26]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [27]:
#lr_model = LinearRegression(n_jobs=-1)
#lr_model.fit(X_train, y_train)

In [28]:
##lr_train_pred = lr_model.predict(X_train)
##lr_test_pred = lr_model.predict(X_test)

In [29]:
#X_train.unique()

In [30]:
#lr_train_pred

In [31]:
#print('Train rmse:', np.sqrt(mean_squared_error(y_train, lr_train_pred)))
#print('Validation rmse:', np.sqrt(mean_squared_error(X_test, lr_test_pred)))

In [32]:

#def fetch_logged_data(run_id):
#    client = mlflow.tracking.MlflowClient()
#    data = client.get_run(run_id).data
#    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
#    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
#    return data.params, data.metrics, tags, artifacts
#
## enable autologging
#mlflow.sklearn.autolog()
##
### prepare training data
#X = X_df['monthly premium auto'].values.reshape(-1,1)
#y = X_df['customer lifetime value'].values.reshape(-1,1)
### train a model
#model = LinearRegression()
#with mlflow.start_run() as run:
#    model.fit(X, y)
##
### fetch logged data
#params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)
##
#pprint(params)
##
#pprint(metrics)
#pprint(tags)
#pprint(artifacts)
#

In [33]:
#with mlflow.start_run(run_name="LinearRegression"):
#    logreg =  LinearRegression()
#    lr = logreg.fit(X_train, y_train)   
#    y_pred_tr = logreg.predict(X_train)
#    mse_train = metrics.mean_absolute_error(y_train, y_pred_tr) 
#   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
#   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)
#
#    y_pred = logreg.predict(y_test)
#    mse_test = metrics.mean_absolute_error(y_test, y_pred) 
#    mlflow.log_metric("train_mse", mse_train)
#    mlflow.log_metric("test_mse", mse_test)
#    mlflow.sklearn.log_model(logreg, f"linear_regression")
    
with mlflow.start_run(run_name="linear_regression"):
    lireg = LinearRegression()
    lr1 = lireg.fit(X_train, y_train)   
    y_pred_tr = lireg.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = lireg.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)


    mlflow.sklearn.log_model(lireg, f"linear_regression")
        

2021/08/04 20:02:40 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



In [34]:
#predictions_train = logreg.predict(X_train)
#predictions = logreg.predict(X_test)
##print('\033[1mTraining Accuracy Percentage\033[0m: %.2f' % (accuracy_score(y_train, predictions_train)*100))
##fpr, tpr, thresholds = metrics.roc_curve(y_train, predictions_train)
##print('\033[1mTraining Auc score\033[0m: %.2f' % (metrics.auc(fpr, tpr)))
#
#print('\033[1mTesting Accuracy Percentage\033[0m: %.2f' % (mean_absolute_error(y_test, predictions)*100))
#fpr, tpr, thresholds = metrics.mean_absolute_error(y_test, predictions)
#print('\033[1mTesting Auc score\033[0m: %.2f' % (metrics.mean_absolute_error(fpr, tpr)))
#



In [35]:
with mlflow.start_run(run_name="knn_"):
    knn = KNeighborsRegressor(n_neighbors=8)
    lr1 = knn.fit(X_train, y_train)   
    y_pred_tr = knn.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = knn.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)
    y_pred = knn.predict(X_test)

    mlflow.sklearn.log_model(knn, f"knn_")

In [36]:
r2_test

0.06384578326880053

In [37]:
y_pred


array([ 9469.53683287,  5667.82680825, 12145.58413562, ...,
        8422.410018  ,  6486.797843  ,  5998.17587725])

In [38]:
y_test

2303    33873.181230
2559     2842.025913
6975     9384.654502
6295     3600.934326
4843    16979.797570
            ...     
1294     5917.466603
6947    13448.809730
8589    10408.968650
5221     8923.154803
7688     6044.020038
Name: customer lifetime value, Length: 1627, dtype: float64

In [39]:
with mlflow.start_run(run_name="BayesianRidge_"):
    clf = BayesianRidge(compute_score=True)  
    lr1 = clf.fit(X_train, y_train)   
    y_pred_tr = lr1.predict(X_train)
    r2_train = r2_score(y_train, y_pred_tr)

    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
    rmse_train1 = math.sqrt(mse_train1)
   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)

    y_pred = lr1.predict(X_test)
    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
    rmse_test1 = math.sqrt(mse_test1)
    r2_test= r2_score(y_test,y_pred)

    
    mlflow.log_metric("train_mse", mse_train1)
    mlflow.log_metric("test_mse", mse_test1)
    mlflow.log_metric("r2_train", r2_train)
    mlflow.log_metric("r2_test", r2_test)


    mlflow.sklearn.log_model(clf, f"BayesianRidge_")

In [40]:
rmse_test1

62.18923742171308

In [41]:
# Select the run of the experiment
df_runs = mlflow.search_runs(experiment_ids=exp_id)
print("Number of runs done : ", len(df_runs))

df_runs

Number of runs done :  60


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_mse,metrics.train_mse,metrics.r2_test,metrics.r2_train,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.type
0,2608f058d11c475cbef14e7ccef91b0b,75,FINISHED,s3://models/75/2608f058d11c475cbef14e7ccef91b0...,2021-08-04 20:02:41.083000+00:00,2021-08-04 20:02:41.275000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""2608f058d11c475cbef14e7ccef91b0b""...",root,LOCAL
1,5293779e61124bc7991d55fcf1a98d34,75,FINISHED,s3://models/75/5293779e61124bc7991d55fcf1a98d3...,2021-08-04 20:02:40.800000+00:00,2021-08-04 20:02:41.031000+00:00,4339.500222,3737.206285,0.063846,0.247271,knn_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""5293779e61124bc7991d55fcf1a98d34""...",root,LOCAL
2,e3f9802c75bb448982d007f7bcc2532d,75,FINISHED,s3://models/75/e3f9802c75bb448982d007f7bcc2532...,2021-08-04 20:02:40.458000+00:00,2021-08-04 20:02:40.765000+00:00,3867.325363,3882.434050,0.185325,0.154231,linear_regression,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""e3f9802c75bb448982d007f7bcc2532d""...",root,LOCAL
3,f2a211a4827f4f1a9deb8b2894f6eeaa,75,FINISHED,s3://models/75/f2a211a4827f4f1a9deb8b2894f6eea...,2021-08-04 19:28:07.913000+00:00,2021-08-04 19:28:08.100000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""f2a211a4827f4f1a9deb8b2894f6eeaa""...",root,LOCAL
4,79f24df2c1d444328e058fec85b731f4,75,FINISHED,s3://models/75/79f24df2c1d444328e058fec85b731f...,2021-08-04 19:28:07.637000+00:00,2021-08-04 19:28:07.853000+00:00,4339.500222,3737.206285,0.063846,0.247271,knn_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""79f24df2c1d444328e058fec85b731f4""...",root,LOCAL
5,33ce1e8f7d294f0eaada46997f209c4a,75,FINISHED,s3://models/75/33ce1e8f7d294f0eaada46997f209c4...,2021-08-04 19:28:07.287000+00:00,2021-08-04 19:28:07.596000+00:00,3867.325363,3882.434050,0.185325,0.154231,linear_regression,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""33ce1e8f7d294f0eaada46997f209c4a""...",root,LOCAL
6,f4c8c855ce604387a505411891274025,75,FINISHED,s3://models/75/f4c8c855ce604387a50541189127402...,2021-08-04 15:14:53.394000+00:00,2021-08-04 15:14:53.602000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""f4c8c855ce604387a505411891274025""...",root,LOCAL
7,b79d064dd19c4f14bc54ee2ddb1a122d,75,FINISHED,s3://models/75/b79d064dd19c4f14bc54ee2ddb1a122...,2021-08-04 15:14:53.152000+00:00,2021-08-04 15:14:53.344000+00:00,4339.500222,3737.206285,0.063846,0.247271,knn_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""b79d064dd19c4f14bc54ee2ddb1a122d""...",root,LOCAL
8,e60dcfa6c9294c5fb42f5cba50881ce8,75,FINISHED,s3://models/75/e60dcfa6c9294c5fb42f5cba50881ce...,2021-08-04 15:14:52.832000+00:00,2021-08-04 15:14:53.117000+00:00,3867.325363,3882.434050,0.185325,0.154231,linear_regression,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""e60dcfa6c9294c5fb42f5cba50881ce8""...",root,LOCAL
9,060bdb8e786842e9978bf16e358a72f4,75,FINISHED,s3://models/75/060bdb8e786842e9978bf16e358a72f...,2021-08-04 14:48:43.450000+00:00,2021-08-04 14:48:43.656000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""060bdb8e786842e9978bf16e358a72f4""...",root,LOCAL


In [42]:
top_runs = df_runs.loc[0:4].sort_values(['metrics.test_mse'],ascending=False)
top_runs.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_mse,metrics.train_mse,metrics.r2_test,metrics.r2_train,tags.mlflow.runName,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.type
1,5293779e61124bc7991d55fcf1a98d34,75,FINISHED,s3://models/75/5293779e61124bc7991d55fcf1a98d3...,2021-08-04 20:02:40.800000+00:00,2021-08-04 20:02:41.031000+00:00,4339.500222,3737.206285,0.063846,0.247271,knn_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""5293779e61124bc7991d55fcf1a98d34""...",root,LOCAL
4,79f24df2c1d444328e058fec85b731f4,75,FINISHED,s3://models/75/79f24df2c1d444328e058fec85b731f...,2021-08-04 19:28:07.637000+00:00,2021-08-04 19:28:07.853000+00:00,4339.500222,3737.206285,0.063846,0.247271,knn_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""79f24df2c1d444328e058fec85b731f4""...",root,LOCAL
0,2608f058d11c475cbef14e7ccef91b0b,75,FINISHED,s3://models/75/2608f058d11c475cbef14e7ccef91b0...,2021-08-04 20:02:41.083000+00:00,2021-08-04 20:02:41.275000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""2608f058d11c475cbef14e7ccef91b0b""...",root,LOCAL
3,f2a211a4827f4f1a9deb8b2894f6eeaa,75,FINISHED,s3://models/75/f2a211a4827f4f1a9deb8b2894f6eea...,2021-08-04 19:28:07.913000+00:00,2021-08-04 19:28:08.100000+00:00,3867.501251,3882.437080,0.185231,0.154229,BayesianRidge_,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""f2a211a4827f4f1a9deb8b2894f6eeaa""...",root,LOCAL
2,e3f9802c75bb448982d007f7bcc2532d,75,FINISHED,s3://models/75/e3f9802c75bb448982d007f7bcc2532...,2021-08-04 20:02:40.458000+00:00,2021-08-04 20:02:40.765000+00:00,3867.325363,3882.434050,0.185325,0.154231,linear_regression,/opt/miniconda/lib/python3.7/site-packages/ipy...,"[{""run_id"": ""e3f9802c75bb448982d007f7bcc2532d""...",root,LOCAL


In [43]:
artifacts = top_runs.iloc[0]["artifact_uri"]
run_id = top_runs.iloc[0]["run_id"]
model_name = top_runs.iloc[0]["tags.mlflow.runName"] 


print('Best model_artifacts :',artifacts)
print("=" * 100)
print('Best model run_id :',run_id)
print("=" * 100)
print('Best model :',model_name)
print("=" * 100)
print("Best model experiment id :",exp_id)

Best model_artifacts : s3://models/75/5293779e61124bc7991d55fcf1a98d34/artifacts
Best model run_id : 5293779e61124bc7991d55fcf1a98d34
Best model : knn_
Best model experiment id : 75


In [44]:
result = mlflow.register_model(
    "runs:/" + run_id,
    MODEL_NAME+'_'+model_name
)

Registered model 'customer-lifetime-value_knn_' already exists. Creating a new version of this model...
2021/08/04 20:02:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: customer-lifetime-value_knn_, version 4
Created version '4' of model 'customer-lifetime-value_knn_'.


In [45]:
print('Registered model information :')
print('=='*50)
print(pd.DataFrame(result, columns =['Attribute','Value']))

Registered model information :
                 Attribute                                              Value
0       creation_timestamp                                      1628107361486
1            current_stage                                               None
2              description                                                   
3   last_updated_timestamp                                      1628107361486
4                     name                       customer-lifetime-value_knn_
5                   run_id                   5293779e61124bc7991d55fcf1a98d34
6                 run_link                                                   
7                   source  s3://models/75/5293779e61124bc7991d55fcf1a98d3...
8                   status                                              READY
9           status_message                                                   
10                    tags                                                 {}
11                 user_id       

In [46]:
print('Experiment information :')
print('=='*50)
print(pd.DataFrame(exp_details, columns =['Attribute','Value']))

Experiment information :
           Attribute           Value
0  artifact_location  s3://models/75
1      experiment_id              75
2    lifecycle_stage          active
3               name   cust-lifetime
4               tags              {}


#lr_features = ['monthly premium auto','customer lifetime value']
#lr_train = X_train[lr_features]
#lr_val = y_validation[lr_features]
#lr_test = y_test[lr_features]

transit model to production

In [47]:
#production_model = client.transition_model_version_stage(
#     name='telecom_customer_churn_logistic_regression',
#     version=3,
#     stage="Production")

In [48]:
#print('Production model information :')
#print('=='*50)
#print(pd.DataFrame(result, columns =['Attribute','Value']))

In [49]:
time = pd.to_datetime(datetime.now())

In [50]:
dictionary = [{"model":model_name,
             "location":artifacts+'/'+model_name,
             "run_id":run_id,
            'experiment_id':exp_id,
              'prod_time':time,
            'top_features':k_best_features,
              }]

In [51]:
dataset = pd.DataFrame(dictionary)
dataset

,model,location,run_id,experiment_id,prod_time,top_features
0,knn_,s3://models/75/5293779e61124bc7991d55fcf1a98d3...,5293779e61124bc7991d55fcf1a98d34,75,2021-08-04 20:02:41.549453,"[income, monthly premium auto, total claim amo..."


In [52]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
PP=write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,TABLE_MODEL)

In [53]:
read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL)
#read_from_feature_store('POSTGRES_PASSWORD','POSTGRES_ADDRESS','POSTGRES_PORT','POSTGRES_DBNAME','PROJECT_SCHEMA','TABLE_NAME')

,model,location,run_id,experiment_id,prod_time,top_features
0,knn_,s3://models/75/5293779e61124bc7991d55fcf1a98d3...,5293779e61124bc7991d55fcf1a98d34,75,2021-08-04 20:02:41.549453,"{income,""monthly premium auto"",""total claim am..."


In [54]:
#cat_features = [0, 1, 7, 8]
#
#catboost_model = CatBoostRegressor(
#    iterations=500,
#    max_ctr_complexity=4,
#    random_seed=0,
#    od_type='Iter',
#    od_wait=25,
#    verbose=50,
#    depth=4
#)
#
#catboost_model.fit(
#    X_train, y_train,
#    cat_features=cat_features,
#    eval_set=(X_test, y_test)
#)

In [55]:
#with mlflow.start_run(run_name="knn"):
#    knn = KNeighborsClassifier(n_neighbors=8)
#    lr = logreg.fit(X_train, y_train)   
#    y_pred_tr = logreg.predict(X_train)
#    mse_train1 = metrics.mean_absolute_error(y_train, y_pred_tr) 
#    rmse_train1 = math.sqrt(mse_train1)
#   # mse_test_x = metrics.mean_absolute_error(y_test, y_pred_tr) 
#   # rmse_train = metrics.root_mean_absolute_error(y_train, y_pred_tr)
#
#    y_pred = logreg.predict(y_test)
#    mse_test1 = metrics.mean_absolute_error(y_test, y_pred) 
#    rmse_test1 = math.sqrt(mse_test1)
#
#    
#    mlflow.log_metric("train_mse", mse_train)
#    mlflow.log_metric("test_mse", mse_test)
#    mlflow.sklearn.log_model(logreg, f"linear_regression")

In [56]:
#catboost_train_pred = catboost_model.predict(X_train)
##catboost_val_pred = catboost_model.predict(X_validation)
#catboost_test_pred = catboost_model.predict(X_test)
#

In [57]:
#X_train

In [58]:
#print(regressor.intercept_)
#For retrieving the slope:
#print(regressor.coef_)

In [59]:
#X_df.shape

In [60]:
#svm_reg=svm.SVR()
#svm_reg.fit(X_train,y_train)
#y1_svm=svm_reg.predict(X_train)
#y1_svm=list(y1_svm)
#y2_svm=svm_reg.predict(X_test)
#y2_svm=list(y2_svm)
#
#error=0
#for i in range(len(y_train)):
#    error+=(abs(y1_svm[i]-y_train[i])/y_train[i])
#train_error_svm=error/len(y_train)*100
#print("Train error = "+'{}'.format(train_error_svm)+" percent"+" in SVM Regressor")
#
#error=0
#for i in range(len(y_test)):
#    error+=(abs(y2_svm[i]-y_test[i])/y_test[i])
#test_error_svm=error/len(y_test)*100
#print("Test error = "'{}'.format(test_error_svm)+" percent in SVM Regressor")

In [61]:
#reg = linear_model.BayesianRidge()
#reg.fit(X_train,y_train)
#y1_reg=reg.predict(X_train)
#y1_reg=list(y1_reg)
#y2_reg=reg.predict(X_test)
#y2_reg=list(y2_reg)
#
#error=0
#for i in range(len(y_train)):
#    error+=(abs(y1_reg[i]-y_train[i])/y_train[i])
#train_error_bay=error/len(y_train)*100
#print("Train error = "+'{}'.format(train_error_bay)+" percent"+" in Bayesian Regression")
#
#error=0
#for i in range(len(y_test)):
#    error+=(abs(y2_reg[i]-y_test[i])/y_test[i])
#test_error_bay=(error/len(y_test))*100
#print("Test error = "+'{}'.format(test_error_bay)+" percent"+" in Bayesian Regression")

In [62]:
# Use only part of features on linear Regression.


In [63]:
#X_train